In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [7]:
!pip install transformers requests beautifulsoup4 pandas numpy 

In [27]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification #convert string into seq of nos
import torch #extract highest seq
import requests #used to extract webpage
from bs4 import BeautifulSoup #extract the reviews we need
import re #used to extract specific review
import numpy as np
import pandas as pd

In [51]:
tokenizer =  AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [58]:
r = requests.get('https://www.yelp.com/biz/international-sport-motors-san-francisco?override_cta=Get+pricing')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [59]:
reviews


['I had some concerns about my car, so I took it into ISM to have it inspected.The team checked it over, and discovered a plastic bag wrapped around the exhaust pipe.  No wonder I was smelling burnt plastic!  My brakes are fine, tires are good, and I feel comfortable on the road again.  Thanks to everyone at ISM for another great experience.',
 'Brought my Lexus NX300h in for its 20k service. Very responsive, professional and reasonably priced. Will definitely use this business in the future. Easy 5 stars',
 'Horrible service, incompetent owner. If you\'re looking for a new shop to take your car in for service/repairs, look elsewhere. Unless you want to deal with mediocre service, *do not* go here.We took our Volkswagen in for a service at this place, which turned out to be a big mistake. On our way home after the service, the tire warning appeared and we contacted the shop to understand why. We were told that the tire pressure light may not have been reset after they set the tire pres

In [60]:
df = pd.DataFrame(np.array(reviews), columns=['review'])


In [61]:
df['review'].iloc[0]

'I had some concerns about my car, so I took it into ISM to have it inspected.The team checked it over, and discovered a plastic bag wrapped around the exhaust pipe.  No wonder I was smelling burnt plastic!  My brakes are fine, tires are good, and I feel comfortable on the road again.  Thanks to everyone at ISM for another great experience.'

In [62]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [63]:
sentiment_score(df['review'].iloc[1])
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))


In [64]:
df

,review,sentiment
0,"I had some concerns about my car, so I took it...",5
1,Brought my Lexus NX300h in for its 20k service...,5
2,"Horrible service, incompetent owner. If you're...",1
3,"Kevin,I’m sorry you had a poor experience here...",2
4,I first want to say how very difficult a task ...,5
5,Their team is super responsive and kind. I sen...,5
6,Great customer service! I had a Catalytic Con...,5
7,Good place for ur car to be serviced and taken...,4
8,Great experience with ISM on my Hyundai Genesi...,5
9,I had dealt with ISM in the past and got great...,5


In [65]:
df.tail(5)

,review,sentiment
7,Good place for ur car to be serviced and taken...,4
8,Great experience with ISM on my Hyundai Genesi...,5
9,I had dealt with ISM in the past and got great...,5
10,"Giovanni and Antonio,I just wanted to say what...",5
11,Today it's been three months since they have h...,1
